# Chapter 05: Coding by Composing

### Defintion of Composing concept 
The composition of two functions `f` and `g` returns a new function `h`, such that:  
`h = f(g) := f ∘ g`

---
#### Functional husbandry

In [1]:
compose = lambda f,g: lambda x: f(g(x))

In [2]:
toUpperCase = lambda x: x.upper()
exclaim = lambda x: f'{x}!'
shout = compose(exclaim, toUpperCase)

In [4]:
shout('send in the clowns')

'SEND IN THE CLOWNS!'